In [1]:
import os
import pandas as pd
import numpy as np
import json
import google.generativeai as genai
from servicios.pydantic_model import ProductDimensions
from dotenv import load_dotenv
from tavily import TavilyClient
from servicios.consulta_invoices import trae_invoices
from servicios.consulta_tarificacion import trae_tarifas



/home/mariano/anaconda3/envs/llmops/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = trae_invoices(202511)
tarifas = trae_tarifas()

RMKeyView(['id', 'periodo', 'proveedor', 'track_code', 'ambito', 'tipo_servicio', 'name', 'main_category', 'sub_category', 'category', 'alto', 'ancho', 'largo', 'peso_aforado', 'peso_fisico', 'peso_facturable', 'tarifa'])
[(10846, 202511, 'BoxFast', 'BFX42754874321X', 2, '24hs', 'SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards', 'car & motorbike', 'Car Parts', 'Car_Parts', Decimal('32.00'), Decimal('18.00'), Decimal('69.00'), Decimal('13.25'), Decimal('12.20'), Decimal('13.25'), Decimal('5600.00')), (10847, 202511, 'BoxFast', 'BFX45088662684X', 2, '24hs', 'ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms', 'car & motorbike', 'Car Parts', 'Car_Parts', Decimal('67.00'), Decimal('50.00'), Decimal('30.00'), Decimal('33.50'), Decimal('19.77'), Decimal('33.50'), Decimal('10100.00')), (10848, 202511, 'BoxFast', 'BFX45832574342X', 2, '72hs', 'AUTOWORLD MAX BOLERO FRONT GRILL WITH CROME CENTRE BLACK FOR MAHINDRA BOLERO 

In [3]:
tarifas['tarifa'] = tarifas['tarifa'].astype(float)
tarifas['ambito'] = tarifas['ambito'].astype(int)
df['tarifa'] = df['tarifa'].astype(float)

In [15]:
df.periodo.unique()
periodo = df['periodo'].unique()[0]
print(f"Periodo: {periodo}")

Periodo: 202511


In [4]:
load_dotenv()
tavily_api_key = os.getenv("TAVILY_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")


In [5]:
tavily_client = TavilyClient(api_key=tavily_api_key)
genai.configure(api_key=google_api_key)

In [6]:
def extraer_datos_con_gemini(contexto: str, nombre_producto: str) -> dict:
    """Usa Gemini para extraer las dimensiones del texto de búsqueda."""
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    prompt = f"""
    Basado en el siguiente contexto de búsqueda para el producto "{nombre_producto}", 
    extrae el alto, ancho, largo y peso.
    
    - Si encuentras las dimensiones exactas, úsalas.
    - Si no las encuentras, busca dimensiones de productos muy similares mencionados en el contexto.
    - Devuelve los valores como números flotantes (float).
    - Tu respuesta DEBE ser únicamente un objeto JSON con las claves "alto", "ancho", "largo", "peso".
    - En la clave "fuente" indica de que pagina web o fuente obtuviste la información.
    - Si no encuentras la fuente, pon "desconocida".
    - Si no puedes determinar alguno de los valores, intenta predecirlo basado en productos similares. determina que tipo de producto es y busca dimensiones típicas.
    - Selecciona la opción más representativa y confiable, utiliza las medidas que a veces suelen estar en el nombre de la publicación o en las imagenes. 
    - Sé especialmente ágil y eficiente al estimar medidas y **sobretodo** peso físico.

    Contexto:
    ---
    {contexto}
    ---
    """
    
    try:
        response = model.generate_content(prompt)
        # Limpiar la respuesta para asegurar que sea un JSON válido
        json_text = response.text.strip().replace("```json", "").replace("```", "")
        return json.loads(json_text)
    except (json.JSONDecodeError, Exception) as e:
        print(f"  - Error al procesar la respuesta de Gemini: {e}")
        return {}


def buscar_dimensiones_producto(nombre_producto: str) -> dict:
    """Usa Tavily para buscar y Gemini para extraer las dimensiones."""
    print(f"\n🔎 Buscando con Tavily: '{nombre_producto}'...")
    
    try:
        # Búsqueda avanzada con Tavily
        search_context = tavily_client.search(
            query=f'dimensions (height, width, length, weight) for product "{nombre_producto}"',
            search_depth="advanced" # Búsqueda más profunda
        )
        
        # Combinamos los resultados para dárselos a Gemini
        contexto_combinado = "\n".join([str(res) for res in search_context['results']])
        
        if not contexto_combinado:
            print("  - Tavily no devolvió resultados.")
            return {}
            
        print("  - Resultados de Tavily obtenidos. Extrayendo con Gemini...")
        datos_extraidos = extraer_datos_con_gemini(contexto_combinado, nombre_producto)
        
        if datos_extraidos:
            print("  - ✅ ¡Extracción con Gemini exitosa!")
            datos_extraidos['fuente'] = "Tavily + Gemini"
            return datos_extraidos
        else:
            print("  - 🟡 Gemini no pudo extraer los datos del contexto.")
            return {}

    except Exception as e:
        print(f"  - ❌ Error durante la búsqueda con Tavily: {e}")
        return {}

In [ ]:
productos = []
for row in df.itertuples():
    print(f"\nProcesando producto: {row.name}")
    dimensiones = buscar_dimensiones_producto(row.name)
    if dimensiones:
        try:
            producto = ProductDimensions(**dimensiones)
            print(f"  - Dimensiones extraídas: {producto}")
            id = row.id
            ambito = row.ambito
            operacion = row.tipo_servicio
            alto_valor = producto.alto
            ancho_valor = producto.ancho
            largo_valor = producto.largo
            peso_valor = producto.peso
            tarifa_proveedor = row.tarifa
            try:
                peso_aforado = ancho_valor * largo_valor * alto_valor / 4000 # Peso aforado en kg
            except Exception as e:
                print(f"  - Error al calcular el peso aforado: {e}")
                peso_aforado = 0
            try:    
                peso_facturable = max(peso_valor, peso_aforado) # Peso facturable en kg
            except Exception as e:
                print(f"  - Error al calcular el peso facturable: {e}")
                peso_facturable = 0
            mask = (
                (tarifas['ambito'] == ambito) & 
                (tarifas['tipo_de_servicio'] == operacion) & 
                (tarifas['rango_desde'] <= peso_facturable) & 
                (tarifas['rango_hasta'] >= peso_facturable)
            )
            
            # Aplicar la máscara y seleccionar la columna 'tarifa'
            resultado_tarifa = tarifas.loc[mask, 'tarifa']

            # Extraer el valor numérico (si se encontró)
            if not resultado_tarifa.empty:
                tarifa_real = resultado_tarifa.iloc[0]
            else:
                tarifa_real = None # O 0, o np.nan, para manejar casos sin tarifa
            diferencia = tarifa_proveedor - tarifa_real if tarifa_real is not None else None

            #print(f"  - Tarifa encontrada: {tarifa_real}")
            datos = {
                "invoice_id": id,
                "nombre_producto": row.name,
                "track_code": row.track_code,
                "alto": alto_valor,
                "ancho": ancho_valor,
                "largo": largo_valor,
                "peso_aforado": peso_aforado,
                "peso_fisico": peso_valor,
                "peso_facturable": peso_facturable,
                "tarifa_proveedor": tarifa_proveedor,
                "tarifa_real": tarifa_real,
                "diferencia": diferencia,
            }
            if tarifa_real < tarifa_proveedor:
                productos.append(datos)
            
        except Exception as e:
            print(f"  - Error al validar las dimensiones con Pydantic: {e}")
    else:
        print("  - No se pudieron extraer dimensiones para este producto.")


Procesando producto: SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards

🔎 Buscando con Tavily: 'SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...


E0000 00:00:1759113937.489615   15923 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=5.0 ancho=15.0 largo=100.0 peso=0.5 fuente='Tavily + Gemini'

Procesando producto: ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms

🔎 Buscando con Tavily: 'ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...
  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=10.0 ancho=0.1 largo=35.0 peso=0.008 fuente='Tavily + Gemini'

Procesando producto: AUTOWORLD MAX BOLERO FRONT GRILL WITH CROME CENTRE BLACK FOR MAHINDRA BOLERO TYPE-3

🔎 Buscando con Tavily: 'AUTOWORLD MAX BOLERO FRONT GRILL WITH CROME CENTRE BLACK FOR MAHINDRA BOLERO TYPE-3'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...
  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=0.5 ancho=0.12 largo=1.25 peso=4.0 fuente='Tavily + Gemini'

Proc

In [13]:
df_resultado = pd.DataFrame(productos)
df_resultado

,invoice_id,nombre_producto,alto,ancho,largo,peso_aforado,peso_fisico,peso_facturable,tarifa_proveedor,tarifa_real,diferencia
0,10846,SDR Rear Bumper Step footstep Trim For Toyota ...,5.00,15.00,100.00,1.875000,0.500,1.87500,5600.0,3400.0,2200.0
1,10847,ISEE 360® Car Bumper Sticker Exterior for Fron...,10.00,0.10,35.00,0.008750,0.008,0.00875,10100.0,3400.0,6700.0
2,10848,AUTOWORLD MAX BOLERO FRONT GRILL WITH CROME CE...,0.50,0.12,1.25,0.000019,4.000,4.00000,6750.0,2250.0,4500.0
3,10850,"Car Power Outlet Receptacle, 12V 24V Auto Ciga...",1.18,1.18,2.56,0.000891,0.120,0.12000,10500.0,2000.0,8500.0
4,10851,Door Sill/Foot Step Stainless Steel Plates wit...,5.00,8.00,28.00,0.280000,0.500,0.50000,16300.0,2800.0,13500.0
